### Lab #2 과제 정답

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
# imblearn의 파이프라인을 사용해야 SMOTE를 적용할 수 있음
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import plotly.express as px

# 데이터 로드 및 기본 준비
path = './WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# customerID는 예측에 사용하지 않으므로 미리 제거
df = df.drop('customerID', axis=1)

X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# [문제 1] 수치형, 범주형 컬럼 식별 정답
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

# [문제 2 & 3] 전처리 파이프라인 정의 정답
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# [문제 4] ColumnTransformer로 통합 정답
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ], remainder='passthrough') # 나머지 컬럼은 그대로 통과

# [문제 5] SMOTE를 포함한 최종 파이프라인 구축 정답
# imblearn.pipeline.Pipeline 사용
final_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))
])

# [문제 6] 파이프라인 훈련 정답
final_pipeline.fit(X_train, y_train)

# [문제 7] 예측 및 평가 정답
y_pred = final_pipeline.predict(X_test)
y_pred_proba = final_pipeline.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"최종 모델 정확도: {accuracy:.4f}")
print(f"최종 모델 ROC-AUC: {roc_auc:.4f}")

# [문제 8] 혼동 행렬 시각화 정답
cm = confusion_matrix(y_test, y_pred)
fig = px.imshow(cm,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['Not Churn', 'Churn'],
                y=['Not Churn', 'Churn'],
                text_auto=True,
                color_continuous_scale='Blues'
               )
fig.update_layout(title_text='Confusion Matrix')
fig.show()

# 결과 분석:
# ROC-AUC 점수가 0.83 이상으로 좋은 예측 성능을 보임.
# 혼동 행렬을 보면, 실제 이탈(Actual Churn) 고객 중 상당수(e.g., 279명)를 예측(Predicted Churn)해냈음(True Positive).
# 이는 SMOTE가 소수 클래스 탐지 성능을 높이는 데 기여했음을 시사함.
# 반면, 이탈하지 않은 고객을 이탈했다고 잘못 예측한 경우(False Positive)도 존재함.